In [92]:
from langchain_ollama import OllamaLLM

from langchain_ollama.llms import OllamaLLM
import ollama
import whisper
import logging

In [ ]:
llm = OllamaLLM(model="dolphin-llama3:8b")

response = llm.invoke("O primeiro homem na lua foi...")
print(response)

## To speech to text

In [ ]:
def speech_to_text(audio_file = "../audio/transcription.mp3", verbose = True):
    with open("../output/video_transcription.txt", "w") as file:
        logging.info('Starting speech to text processing...')
        
        model = whisper.load_model("medium")
        result = model.transcribe(audio_file)
        
        file.write(result["text"])
        
        if verbose:
            print("Text transcription", result["text"])
            
        return result["text"]
    
    
speech_to_text(verbose=False)

In [ ]:
template = (
    "Fornecemos as informações de contexto abaixo. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "A partir desta informação, por favor responda a pergunta: {query_str}\n"
)
qa_template = PromptTemplate(template)


query_data = "Baseando nos seus conhecimentos sobre desenvolvimento de soluções web, \
    por favor gere um planejamento para o layout de uma tela de aplicativo que compusesse todas as informações \
    mencionadas nessa conversa para um novo aplicatico contendo os pontos mencionados"
    
prompt = qa_template.format(context_str=text_to_prompt, query_str="O que você melhoraria no texto para ser utilizado como prompt?")

response = ollama.chat(model="llama3.1", messages=[{
        'role': 'user',
        'content': prompt,
        },
])

print(response['message']['content'])

In [ ]:
template = (
    "Fornecemos as informações de contexto abaixo. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "A partir desta informação, por favor responda a pergunta: {query_str}\n"
)

query_data_main = "Você poderia organizar o texto?"
    
query_data_main_goal = "Baseando nos seus conhecimentos sobre desenvolvimento de soluções web, \
    por favor gere um planejamento para o layout de uma tela de aplicativo que compusesse todas as informações \
    mencionadas nessa conversa para um novo aplicatico contendo os pontos mencionados?"

test = ""    
    
qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
with open("../output/transcription.txt", "r") as transcription:
    text_to_prompt = transcription.read()
    
    sumary_prompt = qa_template.format(context_str=text_to_prompt, query_str=query_data_main)
    sumary_response = ollama.chat(model="dolphin-llama3:8b", messages=[{
            'role': 'user',
            'content': prompt,
         },
    ])
    
    """ planning_prompt = qa_template.format(context_str=sumary_response['message']['content'], query_str=query_data_main_goal)
    planning_response = ollama.chat(model="dolphin-llama3:8b", messages=[{
            'role': 'user',
            'content': prompt,
         },
    ]) """
    
    print(sumary_response['message']['content'])
    
    # print(planning_response['message']['content'])
    

In [ ]:
with open("../output/transcription.txt", "r") as transcription:
    logging.info('Language correction...')
    text_to_prompt = transcription.read()
    
    prompt = f"""Você está corrigindo textos como um professor. Corrija os erros de português no texto a seguir modificando o que for necessário, mas mantendo a mesma estrutura. Segue o texto a ser modificado: {text_to_prompt}"""
                 
    response = ollama.chat(model="llama3.1", messages=[{
            'role': 'user',
            'content': prompt,
         },
    ])
    
    print(prompt)
    
    with open("../output/meeting_transcription.txt", "w") as video_discription:
        video_discription.write(response['message']['content'])

## Prompt Templates

In [ ]:
template = """Question: {question}
Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

## To auto video description based on audio transcription

In [120]:
with open("../output/video_transcription.txt", "r") as transcription:
    logging.info('Creating descriptions...')
    text_to_prompt = transcription.read()
    
    prompt = f"""Você é um criador de conteúdo expert em criar descrições engajadoras para vídeos a partir de textos.\
                Crie três descrições adaptadas para plataformas de vídeo em português-BR sendo que a primeira descrição deve ser generalista,\
                a segunda direta e a terceira descritiva. 
                Texto para descrever: {text_to_prompt}\
                
                RESPOSTA:   
                Descrição 1 ...
                
                Descrição 2 ...
                
                Descrição 3 ...
                """

    response = ollama.chat(model="dolphin-llama3:8b", messages=[{
            'role': 'user',
            'content': prompt,
         },
    ])
    
    data_response = response['message']['content']
    
    
    logging.info('Language correction...')
    with open("../output/video_transcription.txt", "w") as video_discription:
        logging.info('Creating file...')
        
        video_discription.write(response['message']['content'])
        
        logging.info('Done!')

## Geração de um resumo com os pontos importantes

In [ ]:
transcription_text = speech_to_text(audio_file = "../audio/2024_08_21_12_36_53paulo.mp3", verbose=False)

In [86]:
with open("../output/transcription.txt", "r") as transcription:
    logging.info('Creating an overview...')
    text_to_prompt = transcription.read()
    
    prompt = f"""Você é um especialista em desenvolvimento de apps e sites e está em uma reunião. Gere um resumo sobre a transcrição da reunião a seguir elencando os prontos principais do seu ponto de vista. 
                                
                Transcrição da reunião: {text_to_prompt}  
                
                Resposta: Aqui esta o resumo... 
                """
                
    response = ollama.chat(model='llama3.1', messages=[{
            'role': 'user',
            'content': prompt,
         },
    ])
    
    """  data_response = response['message']['content']
    
    prompt = f'Corrija os erros de português no texto a seguir: {data_response}'
    response = ollama.chat(model='llama3.1', messages=[{
            'role': 'user',
            'content': prompt,
         },
    ])
    """
    
    with open("../output/video_transcription.txt", "w") as video_discription:
        logging.info('Creating file...')
        video_discription.write(response['message']['content'])